In [1]:
from symred.symplectic import PauliwordOp
from symred.projection.cs_vqe import CS_VQE

In [2]:
H_full = {
 # noncon
 'IIYI': 0.6,
 'XYXI': 0.7,
 'XZXI': 0.7,
 'XZZI': 0.6,
 'YXYI': 0.1,
#  'ZYZI': 0.6,
#  'ZZXI': 0.1,
 'ZZZI': 0.7,
 'IIIZ': 0.5,
    
 # con
 'XXXI': 0.1,
 'XXYI': 0.5,
 'XXZI': 0.2,
 'YXXI': 0.2,
#  'YYXI': 0.2,
 'YYZI': 0.2,
 'YZXI': 0.1,
 'ZYYI': 0.1
      }

In [3]:
H = PauliwordOp(H_full)

In [4]:
cs_vqe_obj = CS_VQE(H)

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symred/projection/cs_vqe.py:198: ComplexWarning: Casting complex values to real discards the imaginary part
  pauli_mult_signs[index] = gen_mult.coeff_vec[0]


In [5]:
# H_con
cs_vqe_obj.contextual_operator

0.100+0.000j XXXI +
0.200+0.000j YXXI +
0.100+0.000j YZXI +
0.200+0.000j XXZI +
0.500+0.000j XXYI +
0.200+0.000j YYZI +
0.100+0.000j ZYYI

In [6]:
# H_noncon
cs_vqe_obj.noncontextual_operator

0.700+0.000j XZXI +
0.700+0.000j XYXI +
0.600+0.000j IIYI +
0.100+0.000j YXYI +
0.600+0.000j XZZI +
0.700+0.000j ZZZI +
0.500+0.000j IIIZ

In [7]:
#
cs_vqe_obj.decomposed

{'symmetry': 0.500+0.000j IIIZ,
 'clique_0': 0.600+0.000j IIYI +
 0.100+0.000j YXYI,
 'clique_1': 0.700+0.000j XYXI +
 0.600+0.000j XZZI,
 'clique_2': 0.700+0.000j XZXI +
 0.700+0.000j ZZZI}

In [8]:
# A(r)
cs_vqe_obj.clique_operator

-0.253 IIYI +
-0.658 XYXI +
-0.709 XZXI

In [9]:
# q_vec G
cs_vqe_obj.symmetry_generators

-1 YIYI 
 1 IXYI 
-1 IIIZ

In [10]:
# E_noncon
cs_vqe_obj.noncontextual_energy

-2.474841765813149

In [25]:
cs_vqe_obj.C0

1.000+0.000j IIYI

In [24]:
stabs = PauliwordOp({'IIIZ': 1})
H_reduced = cs_vqe_obj.project_onto_subspace(stabilizers= stabs,
                                             enforce_clique_operator= True)

ValueError: attempt to get argmin of an empty sequence

In [17]:
H_reduced

-0.500+0.000j III +
0.100+0.000j XXX +
0.200+0.000j YXX +
0.700+0.000j XZX +
0.700+0.000j XYX +
0.100+0.000j YZX +
0.200+0.000j XXZ +
0.600+0.000j IIY +
0.500+0.000j XXY +
0.100+0.000j YXY +
0.600+0.000j XZZ +
0.700+0.000j ZZZ +
0.200+0.000j YYZ +
0.100+0.000j ZYY